In [1]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [2]:

def fix_type(floatval):
    return int(str(floatval).replace(".", ""))

In [3]:
approval_data = pd.read_csv('data/approval_topline.csv')
approval_data = approval_data[approval_data["subgroup"]=="All polls"]

training_data, testing_data = train_test_split(approval_data, test_size=0.2, random_state=1)


In [4]:

for col in training_data.columns[4:-1]:
    training_data[col] = training_data[col].apply(fix_type)

training_data['date'] = training_data['modeldate'].apply(lambda x: pd.to_datetime(x, dayfirst=True, format="%m/%d/%Y"))
training_data['year'] = training_data['date'].apply(lambda x: x.year)
training_data['month'] = training_data['date'].apply(lambda x: x.month)
training_data['day'] = training_data['date'].apply(lambda x: x.day)

X = training_data.drop(columns=['approve_estimate', 'president', 'subgroup', 'modeldate', 'timestamp', 'date'])
y = training_data['approve_estimate'].apply(fix_type)
X

,approve_hi,approve_lo,disapprove_estimate,disapprove_hi,disapprove_lo,year,month,day
242,47379959,37003116,51478173,57131323,45825023,2022,1,15
140,45870103,37232849,5311246,58075255,48149666,2022,2,18
656,51887455,42546311,47450365,52968756,41931974,2021,8,30
257,47243485,38948759,51528642,57475757,45581528,2022,1,10
1289,58884888,49677542,34557801,39161473,29954128,2021,1,31
...,...,...,...,...,...,...,...,...
767,5732853,47458804,42728752,4738388,38073625,2021,7,24
218,46785888,37849765,53365934,58544633,48187235,2022,1,23
1190,58418726,48696514,38642223,43994073,33290373,2021,3,5
707,54625997,45307378,43754522,49286955,3822209,2021,8,13


In [5]:
for col in testing_data.columns[4:-1]:
    testing_data[col] = testing_data[col].apply(fix_type)

testing_data['date'] = testing_data['modeldate'].apply(lambda x: pd.to_datetime(x, dayfirst=True, format="%m/%d/%Y"))
testing_data['year'] = testing_data['date'].apply(lambda x: x.year)
testing_data['month'] = testing_data['date'].apply(lambda x: x.month)
testing_data['day'] = testing_data['date'].apply(lambda x: x.day)

tests_df = testing_data.drop(columns=['approve_estimate', 'president', 'subgroup', 'modeldate', 'timestamp', 'date'])


In [6]:
model = DecisionTreeClassifier()

model.fit(X, y)
#prediction = model.predict([[45883725,37321164,52726759, 57032258,4842126,2022,4,13]])
predictions = model.predict(tests_df.values)

c:\users\pc\pycharmprojects\approvalratingpredictor\venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


In [7]:
predictions = [float(str(p)[:2]+"."+str(p)[2:]) for p in predictions]

In [8]:
np.mean([(test-prediction) for test, prediction in zip(testing_data['approve_estimate'], predictions)])

0.10958270454545449